# 

## 데이터 불러오기

In [1]:
#데이터 프레임을 다룰 수 있는 패키지를 불러온다.
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
#데이터 불러오기.
m20 = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_m20.sas7bdat') #m30 의과_보건기관의 일반내역을 뽑아온다.
m30 = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_m30.sas7bdat') #m30 의과_보건기관의 진료내역을 뽑아온다.
m60 = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_m60.sas7bdat') #m60 의과_보건기관의 처방내역을 뽑아온다.
p20 = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_p20.sas7bdat') #p20 약국의 명세서를 뽑아온다.
p30 = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_p30.sas7bdat') #p30 약국의 진료내역을 뽑아온다.

bnc = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_bnc.sas7bdat') #성별 보험자구분
bnd = pd.read_sas('C:/Users/yeonuk/Desktop/환자대조군/edu_sample_db/교육용(SAS)/nsc2_edu_bnd.sas7bdat') #출생년도 사망년도

# 출생년도 정보를 숫자로 바꾸고, RN_INDI순으로 정렬 3000명
bnd['BTH_YYYY'] = bnd['BTH_YYYY'].str.decode("utf-8")
bnd['BTH_YYYY'] = bnd['BTH_YYYY'].replace('1921LE','1921') #1921LE 는 그냥 1921로 바꿈.
bnd['BTH_YYYY'] = bnd['BTH_YYYY'].astype(int)
bnd = bnd.sort_values('RN_INDI') 
#bnd.BTH_YYYY : 출생년도, bnc.SEX : 성별 정보를 불러올 것임.
bncd = pd.merge(bnd[['RN_INDI', 'BTH_YYYY']],bnc.drop_duplicates('RN_INDI')[['RN_INDI', 'SEX']])

In [3]:
# 잘 들어 갔는지 확인해 보자. 고혈압 환자의 추출 방법.
m20[(m20['SICK_SYM1']==b'I10') ].info() #주상병 I10(고혈압)을 가진 환자의 행 만 추출  
# 6627 건이있다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6627 entries, 36 to 105708
Data columns (total 16 columns):
RN_INDI               6627 non-null float64
RN_KEY                6627 non-null object
RN_INST               6627 non-null float64
MDCARE_STRT_DT        6627 non-null object
FORM_CD               6627 non-null object
SICK_SYM1             6627 non-null object
SICK_SYM2             4279 non-null object
OPRTN_YN              6627 non-null object
MDCARE_DD_CNT         6627 non-null float64
VSHSP_DD_CNT          6627 non-null float64
TOT_PRSC_DD_CNT       6627 non-null float64
MCARE_RSLT_TYPE       5864 non-null object
ED_RC_TOT_AMT         6627 non-null float64
EDC_SBA               6627 non-null float64
EDC_INSUR_BRDN_AMT    6627 non-null float64
SPCF_SYM_TYPE         9 non-null object
dtypes: float64(8), object(8)
memory usage: 880.1+ KB


In [4]:
m20[(m20['SICK_SYM1']==b'I10')].drop_duplicates('RN_INDI',keep='first').info() 
# RN_INDI 기준으로 중복 제거. 416명의 환자가 있다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416 entries, 36 to 97786
Data columns (total 16 columns):
RN_INDI               416 non-null float64
RN_KEY                416 non-null object
RN_INST               416 non-null float64
MDCARE_STRT_DT        416 non-null object
FORM_CD               416 non-null object
SICK_SYM1             416 non-null object
SICK_SYM2             228 non-null object
OPRTN_YN              416 non-null object
MDCARE_DD_CNT         416 non-null float64
VSHSP_DD_CNT          416 non-null float64
TOT_PRSC_DD_CNT       416 non-null float64
MCARE_RSLT_TYPE       379 non-null object
ED_RC_TOT_AMT         416 non-null float64
EDC_SBA               416 non-null float64
EDC_INSUR_BRDN_AMT    416 non-null float64
SPCF_SYM_TYPE         1 non-null object
dtypes: float64(8), object(8)
memory usage: 55.2+ KB


# 모집단 만들기

### 1. 약물 파일에서 원하는 약물 주성분 코드 불러내기

In [5]:
drug_code = pd.read_excel('C:/Users/yeonuk/Desktop/환자대조군/4_참고 약제급여목록표(2019.08.01.)_ATC매핑.xlsx', # write your directory here
                          sheet_name = 'ATC매핑결과',
                          header = 0                          
                         )
antiHTN = drug_code[(drug_code['식약분류']==213) | (drug_code['식약분류']==214)]

antiHTN.head(5) # 윗쪽 5줄만 확인해 본다. 잘 들어갔는지

,연번,식약분류,주성분코드,제품코드,키값,제품명,업체명,ATC코드,ATC코드 명칭,비고
4487,5187,213,101501ATB,645304940,64530494,아세타졸정(아세타졸아미드)_(0.25g/1정),한림제약(주),S01EC01,acetazolamide,
4488,5189,213,106901ATB,646800920,64680092,아미로정(염산아미로라이드)_(5mg/1정),건일제약(주),C03DB01,amiloride,
4489,5191,213,112901ATB,647801920,64780192,유레틴정(아조세미드)_(60mg/1정),삼진제약(주),C03CA,"Sulfonamides, plain",
4490,5193,213,163801ATB,653800170,65380017,다이릭스정(푸로세미드)_(40mg/1정),신일제약(주),C03CA01,furosemide,
4491,5194,213,163801ATB,641702750,64170275,후릭스정(푸로세미드)_(40mg/1정),일양약품(주),C03CA01,furosemide,


In [6]:
#원하는 약물군들의 주성분 코드를 모은다.
DU = list(antiHTN[antiHTN.식약분류==213].주성분코드) #이뇨제
ACEi = list(antiHTN[antiHTN.ATC코드.str.startswith('C09AA',na=False)].주성분코드) # ATC코드가 C09AA로 시작하는 ACE inhibitor 단일제들을 뽑았다. 
ARB = list(antiHTN[antiHTN.ATC코드.str.startswith('C09CA',na=False)].주성분코드) # ATC코드가 C09CA로 시작하는 angiotensin 2 antagonist 단일제들을 뽑았다. 
CCB = list(antiHTN[antiHTN.ATC코드.str.startswith('C08CA',na=False)].주성분코드) # ATC코드가 C08CA로 시작하는 Dihydropyridine계 calcium channel blocker 단일제들을 뽑았다. 

# 2제 복합제의 주성분 코드를 뽑는다.
ACEi_DU = list(antiHTN[antiHTN.ATC코드.str.startswith('C09BA',na=False)].주성분코드) # 1개
ACEi_CCB = list(antiHTN[antiHTN.ATC코드.str.startswith('C09BB',na=False)].주성분코드) # 2개
ARB_DU = list(antiHTN[antiHTN.ATC코드.str.startswith('C09DA',na=False)].주성분코드) # 405개
ARB_CCB = list(antiHTN[antiHTN.ATC코드.str.startswith('C09DB',na=False)].주성분코드) #770개

#총 2071개
## 참고. C08G로 시작하는 ATC 코드는 3제병합

In [7]:
# 주성분 코드와 약물군이 표시된 표를 만든다.
DU_table = pd.DataFrame({'GNL_NM_CD': DU, 'Drug_Type': 'DU'})
ACEi_table = pd.DataFrame({'GNL_NM_CD': ACEi, 'Drug_Type': 'ACEi'})
ARB_table = pd.DataFrame({'GNL_NM_CD': ARB, 'Drug_Type': 'ARB'})
CCB_table = pd.DataFrame({'GNL_NM_CD': CCB, 'Drug_Type': 'CCB'})
ACEi_DU_table =  pd.DataFrame({'GNL_NM_CD': ACEi_DU, 'Drug_Type': 'ACEi_DU'})
ACEi_CCB_table = pd.DataFrame({'GNL_NM_CD': ACEi_CCB, 'Drug_Type': 'ACEi_CCB'})
ARB_DU_table = pd.DataFrame({'GNL_NM_CD': ARB_DU, 'Drug_Type': 'ARB_DU'})
ARB_CCB_table = pd.DataFrame({'GNL_NM_CD': ARB_CCB, 'Drug_Type': 'ARB_CCB'})

Drug_table = pd.concat([DU_table, ACEi_table, ARB_table, CCB_table, ACEi_DU_table, ACEi_CCB_table, ARB_DU_table, ARB_CCB_table ])
Drug_table = Drug_table.drop_duplicates()
Drug_Type_list = list(Drug_table.Drug_Type.drop_duplicates()) #

In [8]:
# 연구에 사용할 약물리스트를 엑셀파일로 저장한다.
drug_study = antiHTN[antiHTN.주성분코드.isin(DU+ACEi+ARB+CCB+ACEi_DU+ACEi_CCB+ARB_DU+ARB_CCB)]
drug_study.to_excel('drug.xlsx',sheet_name='Sheet1')

### 2. 고혈압 있는 관측값 뽑아내고, 이전에 outcome 발생한 환자는 제외하여 모집단 만들기

In [9]:
# outcome으로 판단할 코드. .x로 추가되는 하위코드 모두를 포함한다.
CV_event_code = (b'I11',b'I12',b'I13',b'I15',b'I21',b'I22',b'I23',b'I50',b'I60',b'I61',b'I63',b'I64',b'I693',b'G45',b'G46',b'I200',b'I25',b'I48',b'I46')

# 주 or 부상병 I10(고혈압)을 가진 청구항 만을 추출.
Disease_I10 = m20[(m20['SICK_SYM1']==b'I10') | (m20['SICK_SYM2']==b'I10')]	# 8675건이있다. 
 
# m20에서 주,부상병으로 I10을 가지고 있는 청구항을 뽑아서(Disease_I10) 그것과 같은 RN_KEY를 가지고 있는 것을 m60에서 뽑는다.
m60_I10 = m60[m60.RN_KEY.isin(list(Disease_I10.RN_KEY))].sort_values('RN_INDI')#32904건 #502 명

m60_I10['MDCARE_STRT_DT'] = pd.to_datetime(m60_I10['MDCARE_STRT_DT'].str.decode("utf-8")) # 날짜 포맷을 바꿔줌
m60_I10['GNL_NM_CD'] = m60_I10['GNL_NM_CD'].str.decode("utf-8") # 주성분 코드 포맷 바꿔줌. 엑셀에서 가져온 주성분 코드는 string 타입.

#RN_INDI, RN_KEY, MDCARE_STRT_DT, TOT_MCNT, GNL_NM_CD 만 남기고 필요없는 열은 지움
m60_I10 = m60_I10[['RN_INDI',	'RN_KEY',	'MDCARE_STRT_DT',	'TOT_MCNT',	'GNL_NM_CD']]

'''표를 이용하기 좋게 만들기'''
# 1)표에 m20에서 가져온 상병명을 붙이고 개인고유번호로 정렬한다.
m60_I10 = pd.merge(m60_I10, Disease_I10[['RN_KEY','SICK_SYM1','SICK_SYM2']], on='RN_KEY', how='left')
# 2)어떤 약물군인지 표시해 준다. GNL_NM_CD를 참조하여 약물데이터베이스 엑셀 성분명을 가져옴.
m60_I10 = m60_I10.merge(Drug_table, on='GNL_NM_CD',how="left") #관심약물군 이외에는 NaN처리.
# 3) 나이와 성별을 붙여준다.
m60_I10 = m60_I10.merge(bncd, on='RN_INDI',how="left")
m60_I10['AGE'] = m60_I10.MDCARE_STRT_DT.dt.year - m60_I10.BTH_YYYY
del m60_I10['BTH_YYYY']
# 4)개인고유번호, 청구항날짜 순서로 정렬한다.
m60_I10 = m60_I10.set_index(['RN_INDI', 'MDCARE_STRT_DT']).sort_index()
m60_I10 = m60_I10.reset_index()


# m20 데이터에서, CV_event_code를 포함한 관측값만을 뽑고. RN_KEY, SICK_SYM1, SICK_SYM2 3가지 열만 남김
CV_event_KEY = m20[(m20.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (m20.SICK_SYM2.str.startswith(CV_event_code,na=False))]
CV_outcome_overall = m60_I10[m60_I10.RN_KEY.isin(list(CV_event_KEY.RN_KEY))] #3381건 71명
patient_CV_overall = m60_I10[m60_I10.RN_INDI.isin(list(CV_outcome_overall.RN_INDI))] #6706건 71명

CV_overall_ID = list(patient_CV_overall.RN_INDI.drop_duplicates())
exclude_list = []
for p in CV_overall_ID : 
    one_person = patient_CV_overall[patient_CV_overall.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    I10_first_time = one_person[(one_person['SICK_SYM1']==b'I10') | (one_person['SICK_SYM2']==b'I10')].MDCARE_STRT_DT.min()
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() #outcome 발병 최초일
    if I10_first_time > CV_first_time : # outcome이 고혈압 진단시점보다 앞선다. :제외 조건.
        exclude_list.append(p)

#제외 조건을 만족하는 모집단을 만든다.        
m60_I10_main = m60_I10[~m60_I10.RN_INDI.isin(exclude_list)] #32904건 480명

In [10]:
exclude_list #'outcome이 고혈압 진단시점보다 앞선 사람은 없었다 !'

[]

In [11]:
m60_I10_main.set_index(['RN_INDI', 'MDCARE_STRT_DT']).sort_index()

RN_KEY  TOT_MCNT  GNL_NM_CD SICK_SYM1  \
RN_INDI   MDCARE_STRT_DT                                                     
1075059.0 2006-01-04      b'20060000004329'      10.0  107601ATB    b'I10'   
          2006-01-04      b'20060000004329'       7.0  165305ATB    b'I10'   
          2006-01-04      b'20060000004329'      10.0  165305ATB    b'I10'   
          2006-01-04      b'20060000004329'       7.0  107601ATB    b'I10'   
          2006-01-04      b'20060000004329'      10.0  111403ATB    b'I10'   
          2006-01-04      b'20060000004329'       7.0  111403ATB    b'I10'   
          2006-02-03      b'20060000004976'      15.0  107601ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  438901ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  438901ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  281300ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  281300ATB    b'I10'   
          2006-02-03      b'20060000004976'      15.0  107601ATB    b'I10'   
          2006-02-03      b'20060000004976'      15.0  165305ATB    b'I10'   
          2006-02-03      b'20060000004976'      15.0  165305ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  125201ACH    b'I10'   
          2006-02-03      b'20060000004976'       2.0  125201ACH    b'I10'   
          2006-02-03      b'20060000004976'       2.0  459301ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  459301ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  234401ATB    b'I10'   
          2006-02-03      b'20060000004976'       2.0  234401ATB    b'I10'   
          2006-02-03      b'20060000004976'      15.0  111403ATB    b'I10'   
          2006-02-03      b'20060000004976'      15.0  111403ATB    b'I10'   
          2006-03-03      b'20060000010864'       2.0  281300ATB    b'I10'   
          2006-03-03      b'20060000010864'       2.0  125201ACH    b'I10'   
          2006-03-03      b'20060000010864'       2.0  459301ATB    b'I10'   
          2006-03-03      b'20060000010864'       2.0  142902ATB    b'I10'   
          2006-03-03      b'20060000010864'       2.0  234401ATB    b'I10'   
          2006-03-03      b'20060000010864'       2.0  226802ATB    b'I10'   
          2008-01-19      b'20080000003025'      30.0  124001ACH    b'I10'   
          2008-01-19      b'20080000003025'      30.0  222901ATB    b'I10'   
...                                     ...       ...        ...       ...   
9886714.0 2010-10-18      b'20100000056559'       3.0  105203ATB    b'I10'   
          2010-10-18      b'20100000056559'       3.0  125201ACH    b'I10'   
          2010-10-21      b'20100000058910'       4.0  105203ATB    b'I10'   
          2010-10-21      b'20100000058910'       4.0  430600ATB    b'I10'   
          2010-10-21      b'20100000058910'       4.0  438901ATB    b'I10'   
          2010-10-21      b'20100000058910'       4.0  125201ACH    b'I10'   
          2010-11-30      b'20100000062828'      30.0  262500ATB    b'I10'   
          2010-12-23      b'20100000071382'      30.0  262500ATB    b'I10'   
9918716.0 2006-01-09      b'20060000003362'      30.0  459801ATB    b'I10'   
          2006-02-09      b'20060000008413'      30.0  459801ATB    b'I10'   
          2006-03-06      b'20060000011086'      30.0  459801ATB    b'I10'   
          2006-04-12      b'20060000017636'      30.0  459801ATB    b'I10'   
          2006-05-08      b'20060000020414'      30.0  459801ATB    b'I10'   
          2006-06-07      b'20060000027048'      30.0  459801ATB    b'I10'   
          2006-07-07      b'20060000029469'      60.0  459801ATB    b'I10'   
          2006-09-08      b'20060000041080'      60.0  459801ATB    b'I10'   
          2006-11-08      b'20060000047885'      60.0  459801ATB    b'I10'   
          2006-12-13      b'20060000052161'      60.0  459801ATB    b'I10'   
          2008-02-19      b'20080000008280'      60.0  4598

# 베이스라인 비교

In [12]:
import scipy.stats as stats

In [13]:
# m20 데이터에서, CV_event_code를 포함한 관측값만을 뽑고. RN_KEY, SICK_SYM1, SICK_SYM2 3가지 열만 남김
CV_event_KEY = m20[(m20.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (m20.SICK_SYM2.str.startswith(CV_event_code,na=False))]

CV_outcome = m60_I10_main[m60_I10_main.RN_KEY.isin(list(CV_event_KEY.RN_KEY))] #3981건 71명

#위 71명의 RN_INDI를 모수에서 검색하여 각 환자별 전체 진료 내역을 가져온다. 6706건
patient_CV_outcome = m60_I10_main[m60_I10_main.RN_INDI.isin(list(CV_outcome.RN_INDI))]

In [14]:
patient_CV_minus = m60_I10_main[~m60_I10_main.RN_INDI.isin(list(patient_CV_outcome.RN_INDI))] #409명. 26198건
# 관찰기간동안 CV outcome이 없는 환자.
CV_minus_list = list(patient_CV_minus.RN_INDI.drop_duplicates())

In [15]:
DM_code = (b'E10',b'E11',b'E12',b'E13',b'E14')
print('CV+에서 DM 수 :', len(patient_CV_outcome[(patient_CV_outcome.SICK_SYM1.str.startswith(DM_code,na=False)) | (patient_CV_outcome.SICK_SYM2.str.startswith(DM_code,na=False))].RN_INDI.drop_duplicates()))
#CV+ 에서 당뇨 있는사람 11명

print('CV-에서 DM 수 :', len(patient_CV_minus[(patient_CV_minus.SICK_SYM1.str.startswith(DM_code,na=False)) | (patient_CV_minus.SICK_SYM2.str.startswith(DM_code,na=False))].RN_INDI.drop_duplicates())) #.MDCARE_STRT_DT.min()
# CV- 에서 당뇨있는사람 97명 

table = [[11,97],[60,312]]
print(*table,sep ='\n')


chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)

CV+에서 DM 수 : 11
CV-에서 DM 수 : 97
[11, 97]
[60, 312]
chi2 :  1.898285101941142 p-Value: 0.16827039634529525


In [16]:
print('CV+남자 :' ,len(patient_CV_outcome[patient_CV_outcome.SEX == b'1'].RN_INDI.drop_duplicates())) #남자 28명

print('CV - 남자 :', len(patient_CV_minus[patient_CV_minus.SEX == b'1'].RN_INDI.drop_duplicates())) #  남자 167명

table = [[28,167],[43,242]]
print(*table,sep ='\n')


chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)


CV+남자 : 28
CV - 남자 : 167
[28, 167]
[43, 242]
chi2 :  0.008097444829534705 p-Value: 0.9282984962316274


In [17]:
# 나이통계
CV_patients = list(patient_CV_outcome.RN_INDI.drop_duplicates()) 
patient_age = pd.DataFrame({'CV_patient_RN_INDI': CV_patients, 'AGE':0 }).set_index('CV_patient_RN_INDI')


for p in CV_patients :
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() #outcome 발병 최초일
    
    patient_age.loc[p] = one_person[one_person.MDCARE_STRT_DT == CV_first_time].AGE.min() # 처음 CV outcome 나왔을 때 나이

# CV +의 나이 들 
CV_plus_age = list(patient_age.AGE) # 처음 CV outcome 나왔을 때 나이
# CV -의 나이 들 
CV_minus_age = list(patient_CV_minus.drop_duplicates('RN_INDI', keep='last').AGE) # 관측의 마지막날 나이.

st, pvalue = stats.ttest_ind(CV_plus_age,CV_minus_age)
print(st, "p-Value:", pvalue)

2.7942038099138164 p-Value: 0.005411824923689731


In [18]:
print('outcome 발생환자 : ' ,np.mean(CV_plus_age))
print('outcome 발생하지 않은 환자 : ' ,np.mean(CV_minus_age))

outcome 발생환자 :  70.32394366197182
outcome 발생하지 않은 환자 :  65.59902200488997


### Outcome 발생한 환자 분석 CV + 

In [18]:
# m20 데이터에서, CV_event_code를 포함한 관측값만을 뽑고. RN_KEY, SICK_SYM1, SICK_SYM2 3가지 열만 남김
CV_event_KEY = m20[(m20.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (m20.SICK_SYM2.str.startswith(CV_event_code,na=False))]

CV_outcome = m60_I10_main[m60_I10_main.RN_KEY.isin(list(CV_event_KEY.RN_KEY))] #3981건 71명

#위 71명의 RN_INDI를 모수에서 검색하여 각 환자별 전체 진료 내역을 가져온다. 6706건
patient_CV_outcome = m60_I10_main[m60_I10_main.RN_INDI.isin(list(CV_outcome.RN_INDI))]

In [19]:
# 이 중 심혈관계 질병코드(outcome)발생이 관심약물 투여 이후인 환자번호(RN_INDI)를 이 리스트에 저장할 것임
CV_plus_list = [] # Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)
drug_plus_list = [] # outcome에 앞서 사용한 관심약물들을 저장한다.
Less_than_a_week = [] #Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)

CV_patients = list(patient_CV_outcome.RN_INDI.drop_duplicates()) #이 환자군을 돌아가면서 찾을건데
for p in CV_patients :
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    drug_first_time = one_person[one_person.Drug_Type.isin(Drug_Type_list)].MDCARE_STRT_DT.min() #관심약물을 먹은 최초일
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() #outcome 발병 최초일
    
    if drug_first_time < CV_first_time : #약을 처방한 시점이 앞서면 리스트에 저장한다.
        before_CV = one_person[one_person.MDCARE_STRT_DT < CV_first_time]
        
        medi_date = list(before_CV.MDCARE_STRT_DT.drop_duplicates()) 
        minimum_drugdate = 0 
        for d in medi_date : # 약의 처방일수가 7일 미만이면 제외한다.
            one_day = before_CV[before_CV.MDCARE_STRT_DT == d]
            minimum_drugdate+=np.max(list(one_day.TOT_MCNT))
        
        if minimum_drugdate < 7 : 
            Less_than_a_week.append(p)
            continue
        
        CV_plus_list.append(p)
        drug_plus_list.append(list(before_CV.Drug_Type.value_counts().index))
        
drug_plus_stat = patient_CV_outcome[patient_CV_outcome.RN_INDI.isin(CV_plus_list)].Drug_Type.value_counts()
  

print('Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)는 총', len(CV_plus_list), '명 입니다.') # 24명
print('Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)는 총', len(Less_than_a_week), '명 입니다.','\n') # 1명

print('\nOutcome 발생이 관심약물 투여 이후인 환자에서 사용한 개별 약물 성분의 통계 :')
print(drug_plus_stat)


Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)는 총 24 명 입니다.
Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)는 총 1 명 입니다. 


Outcome 발생이 관심약물 투여 이후인 환자에서 사용한 개별 약물 성분의 통계 :
CCB        334
DU         200
ARB        114
ACEi        51
ARB_CCB     16
ARB_DU       9
Name: Drug_Type, dtype: int64


## 환자별 병용 요법의 가지 수를 정의한다. (약물 사용의 변화를 추적할 수 있다.)

In [20]:
# 계속해서, 약물 사용의 변화를 알아본다. (CV+/ outcome이전 약물 사용 환자군)
patient_drug_group = pd.DataFrame({'CV_plus_RN_INDI': CV_plus_list, 'combination_n':0 }).set_index('CV_plus_RN_INDI') #여기에 환자별 병용요법 가지 수를 넣을 것임.

for p in CV_plus_list : 
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    #outcome 발병 최초일
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() 
    # ! 조심 one_person 변수 새로이 할당됨.
    one_person = one_person[one_person.Drug_Type.isin(Drug_Type_list)] #관심약물이 있는 청구항만 고름.
    medi_date = list(one_person.MDCARE_STRT_DT.drop_duplicates()) # 한 환자의 청구항들의 날짜들
    
    
    combi_drugs = [0,0,0,0,0]
    
    print('patient RN_INDI : ',p)
    for d in medi_date : # 날짜별 단위로 묶어서 약물 출력할 것임.
        if d < CV_first_time :
            one_day = one_person[one_person.MDCARE_STRT_DT == d]
            print(list(one_day.Drug_Type), list(one_day.TOT_MCNT), 'days' , '   ->   ', end='')
           
            #### 어떤 병용 요법을 가장 오래 썼는지 통계를 낸다. 
            drugli = list(one_day.Drug_Type)
            for indx in range(len(drugli)):
                drugli[indx] = drugli[indx].split('_')

            ex_list = np.hstack(drugli)
            i = len(list(set(ex_list)))
            combi_drugs[i]+=np.min(list(one_day.TOT_MCNT))  # 한번 처방일에 준 약물 조합들 중 최소 처방일을 그 처방약을 다 먹은 기간으로 정의.
            ####

        else:
            break
    
    patient_drug_group.loc[p] = np.argmax(combi_drugs)
    print('end point')
    print(np.argmax(combi_drugs), '제 요법이 가장 오래 사용됨.')
    print('\n\n')
        

patient RN_INDI :  1144977.0
['CCB'] [60.0] days    ->   ['CCB'] [60.0] days    ->   ['CCB'] [60.0] days    ->   ['CCB'] [60.0] days    ->   ['CCB'] [60.0] days    ->   ['CCB'] [90.0] days    ->   ['CCB'] [60.0] days    ->   ['CCB'] [30.0] days    ->   ['CCB'] [30.0] days    ->   ['CCB', 'ARB'] [30.0, 30.0] days    ->   ['CCB'] [30.0] days    ->   ['CCB'] [30.0] days    ->   end point
1 제 요법이 가장 오래 사용됨.



patient RN_INDI :  1442866.0
['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['ACEi'] [30.0] days    ->   ['CCB'] [7.0] days    ->   end point
1 제 요법이 가장 오래 사용됨.



patient R

In [21]:
'''CV+/약물 사용 환자 중에서 1,2,3제 요법을 사용한 환자의 수.'''
# 7일미만 사용자는 0으로 표시 
for i in range(len(Less_than_a_week)) :
    patient_drug_group.loc[Less_than_a_week[i]] = 0

patient_drug_group = patient_drug_group.sort_index()    
display(patient_drug_group)
print(patient_drug_group.combination_n.value_counts().sort_index())

,combination_n
CV_plus_RN_INDI,
1144977.0,1
1442866.0,1
1727902.0,1
2067949.0,1
2251892.0,1
3162578.0,2
3667656.0,1
3694923.0,0
4418933.0,1


0     1
1    14
2     7
3     3
Name: combination_n, dtype: int64


### Outcome 발생하지 않은 환자 분석 CV -

In [22]:
patient_CV_minus = m60_I10_main[~m60_I10_main.RN_INDI.isin(list(patient_CV_outcome.RN_INDI))] #409명. 26198건
# 관찰기간동안 CV outcome이 없는 환자.
CV_minus_list = list(patient_CV_minus.RN_INDI.drop_duplicates())

In [23]:
patient_CV_minus.Drug_Type.value_counts() #사용한 약물의 통계. 전체적으로 복합제보다는 단일제 투여가 많음.

CCB         3416
ARB_DU      1145
DU          1140
ARB         1012
ACEi         366
ARB_CCB      208
ACEi_CCB       4
Name: Drug_Type, dtype: int64

In [24]:
# 여기에 환자별 병용요법 가지 수를 넣을 것임.
minus_drug_group = pd.DataFrame({'CV_plus_RN_INDI': CV_minus_list, 'combination_n':0 }).set_index('CV_plus_RN_INDI') 


minus_patients = list(patient_CV_minus.RN_INDI.drop_duplicates()) #이 환자군을 돌아가면서 찾을건데
for p in minus_patients : 
    one_person = patient_CV_minus[patient_CV_minus.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.

    if ~(one_person.Drug_Type.isin(Drug_Type_list).any()) : # 관심 혈압강하제가 하나도 없으면 사용약물 0으로 하고 그냥 넘어감
        minus_drug_group.loc[p] = 0 
        continue
    
    one_person = one_person[one_person.Drug_Type.isin(Drug_Type_list)] #관심약물이 있는 청구항만 고름.
    medi_date = list(one_person.MDCARE_STRT_DT.drop_duplicates()) # 한 환자의 청구항들의 날짜들
    combi_drugs = [0,0,0,0,0]
    for d in medi_date : # 날짜별 단위로 묶어서 약물 출력할 것임.
        one_day = one_person[one_person.MDCARE_STRT_DT == d]
        
        #### 어떤 병용 요법을 가장 오래 썼는지 통계를 낸다. 
        drugli = list(one_day.Drug_Type)
        for indx in range(len(drugli)):
            drugli[indx] = drugli[indx].split('_')

        ex_list = np.hstack(drugli)
        i = len(list(set(ex_list)))
        combi_drugs[i]+=np.min(list(one_day.TOT_MCNT)) # 한번 처방일에 준 약물 조합들 중 최소 처방일을 그 처방약을 다 먹은 기간으로 정의.
        ####
    
    minus_drug_group.loc[p] = np.argmax(combi_drugs)
    

print('병용개수   환자수')    
print(minus_drug_group.combination_n.value_counts().sort_index())

병용개수   환자수
0     49
1    218
2    117
3     25
Name: combination_n, dtype: int64


# 통계분석

In [25]:
plus_group_table = patient_drug_group.combination_n.value_counts().sort_index()
plus_group_table.loc[0]=(len(list(patient_CV_outcome.RN_INDI.drop_duplicates())) 
                          - len(CV_plus_list))
plus_group_table =plus_group_table.sort_index()

total_table = pd.merge(plus_group_table,
        minus_drug_group.combination_n.value_counts().sort_index(), 
        left_index=True, right_index=True)
total_table = total_table.rename(columns={'combination_n_x':'CV+','combination_n_y':'CV-'})
total_table

,CV+,CV-
0,47,49
1,14,218
2,7,117
3,3,25


In [26]:

def odds_ratio(table):
    '''
    [[a,b],
    [c,d]]
    의 테이블을 받아서 ad/bc를 도출
    
    '''
    return (table[0][0]*table[1][1])/(table[0][1]*table[1][0])

In [27]:
'''
병용투여	CV outcome +	CV outcome - 
1+2+3	     24	              360
0	         47	               49
'''


table = [list(np.array(total_table.iloc[1])+np.array(total_table.iloc[2])+np.array(total_table.iloc[3])),
        list(total_table.iloc[0])]

print(*table,sep ='\n')


chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)


odds_r = odds_ratio(table)

print("Odds Ratio : ", odds_r )

[24, 360]
[47, 49]
chi2 :  107.78160404972621 p-Value: 3.0008884226616526e-25
Odds Ratio :  0.06950354609929078


In [28]:
'''
병용투여	CV outcome +	CV outcome - 
1    	     14	               218
0	         47	               49
'''

table = [list(total_table.iloc[1]),
        list(total_table.iloc[0])]

print(*table,sep ='\n')


chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)

odds_r = odds_ratio(table)

print("Odds Ratio : ", odds_r )

[14, 218]
[47, 49]
chi2 :  79.82878983238164 p-Value: 4.0829982852244654e-19
Odds Ratio :  0.06695295725161038


In [29]:
'''
병용투여	CV outcome +	CV outcome - 
1    	     14	               218
2	          7            	   117
3	          3              	25
'''

table = [list(total_table.iloc[1]),
        list(total_table.iloc[2]),
        list(total_table.iloc[3])]

print(*table,sep ='\n')


chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)


[14, 218]
[7, 117]
[3, 25]
chi2 :  1.0481911118173632 p-Value: 0.5920906351455624


# 층화 분석

In [19]:
# 모집단을 바꿈. 
m60_I10_main = m60_I10_main[m60_I10.AGE>=65] 

In [20]:
# 위에서 한 분석을 모집단만 바꿔서 그대로 시행한다. 

CV_event_KEY = m20[(m20.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (m20.SICK_SYM2.str.startswith(CV_event_code,na=False))]

CV_outcome = m60_I10_main[m60_I10_main.RN_KEY.isin(list(CV_event_KEY.RN_KEY))] 

patient_CV_outcome = m60_I10_main[m60_I10_main.RN_INDI.isin(list(CV_outcome.RN_INDI))]

# 이 중 심혈관계 질병코드(outcome)발생이 관심약물 투여 이후인 환자번호(RN_INDI)를 이 리스트에 저장할 것임
CV_plus_list = [] # Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)
drug_plus_list = [] # outcome에 앞서 사용한 관심약물들을 저장한다.
Less_than_a_week = [] #Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)

CV_patients = list(patient_CV_outcome.RN_INDI.drop_duplicates()) #이 환자군을 돌아가면서 찾을건데
for p in CV_patients :
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    drug_first_time = one_person[one_person.Drug_Type.isin(Drug_Type_list)].MDCARE_STRT_DT.min() #관심약물을 먹은 최초일
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() #outcome 발병 최초일
    
    if drug_first_time < CV_first_time : #약을 처방한 시점이 앞서면 리스트에 저장한다.
        before_CV = one_person[one_person.MDCARE_STRT_DT < CV_first_time]
        
        medi_date = list(before_CV.MDCARE_STRT_DT.drop_duplicates()) 
        minimum_drugdate = 0 
        for d in medi_date : # 약의 처방일수가 7일 미만이면 제외한다.
            one_day = before_CV[before_CV.MDCARE_STRT_DT == d]
            minimum_drugdate+=np.max(list(one_day.TOT_MCNT))
        
        if minimum_drugdate < 7 : 
            Less_than_a_week.append(p)
            continue
        
        CV_plus_list.append(p)
        drug_plus_list.append(list(before_CV.Drug_Type.value_counts().index))
        
drug_plus_stat = patient_CV_outcome[patient_CV_outcome.RN_INDI.isin(CV_plus_list)].Drug_Type.value_counts()
  

print('Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)는 총', len(CV_plus_list), '명 입니다.') # 
print('Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)는 총', len(Less_than_a_week), '명 입니다.','\n') #

# 계속해서, 약물 사용의 변화를 알아본다. (CV+/ outcome이전 약물 사용 환자군)
patient_drug_group = pd.DataFrame({'CV_plus_RN_INDI': CV_plus_list, 'combination_n':0 }).set_index('CV_plus_RN_INDI') #여기에 환자별 병용요법 가지 수를 넣을 것임.

for p in CV_plus_list : 
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    #outcome 발병 최초일
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() 
    # ! 조심 one_person 변수 새로이 할당됨.
    one_person = one_person[one_person.Drug_Type.isin(Drug_Type_list)] #관심약물이 있는 청구항만 고름.
    medi_date = list(one_person.MDCARE_STRT_DT.drop_duplicates()) # 한 환자의 청구항들의 날짜들
    
    
    combi_drugs = [0,0,0,0,0]
    
    print('patient RN_INDI : ',p)
    for d in medi_date : # 날짜별 단위로 묶어서 약물 출력할 것임.
        if d < CV_first_time :
            one_day = one_person[one_person.MDCARE_STRT_DT == d]

           
            #### 어떤 병용 요법을 가장 오래 썼는지 통계를 낸다. 
            drugli = list(one_day.Drug_Type)
            for indx in range(len(drugli)):
                drugli[indx] = drugli[indx].split('_')

            ex_list = np.hstack(drugli)
            i = len(list(set(ex_list)))
            combi_drugs[i]+=np.min(list(one_day.TOT_MCNT))  # 한번 처방일에 준 약물 조합들 중 최소 처방일을 그 처방약을 다 먹은 기간으로 정의.
            ####

        else:
            break
    
    patient_drug_group.loc[p] = np.argmax(combi_drugs)

        
'''CV+/약물 사용 환자 중에서 1,2,3제 요법을 사용한 환자의 수.'''
# 7일미만 사용자는 0으로 표시 
for i in range(len(Less_than_a_week)) :
    patient_drug_group.loc[Less_than_a_week[i]] = 0

patient_drug_group = patient_drug_group.sort_index()    

patient_CV_minus = m60_I10_main[~m60_I10_main.RN_INDI.isin(list(patient_CV_outcome.RN_INDI))] #409명. 26198건
# 관찰기간동안 CV outcome이 없는 환자.
CV_minus_list = list(patient_CV_minus.RN_INDI.drop_duplicates())

# 여기에 환자별 병용요법 가지 수를 넣을 것임.
minus_drug_group = pd.DataFrame({'CV_plus_RN_INDI': CV_minus_list, 'combination_n':0 }).set_index('CV_plus_RN_INDI') 


minus_patients = list(patient_CV_minus.RN_INDI.drop_duplicates()) #이 환자군을 돌아가면서 찾을건데
for p in minus_patients : 
    one_person = patient_CV_minus[patient_CV_minus.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.

    if ~(one_person.Drug_Type.isin(Drug_Type_list).any()) : # 관심 혈압강하제가 하나도 없으면 사용약물 0으로 하고 그냥 넘어감
        minus_drug_group.loc[p] = 0 
        continue
    
    one_person = one_person[one_person.Drug_Type.isin(Drug_Type_list)] #관심약물이 있는 청구항만 고름.
    medi_date = list(one_person.MDCARE_STRT_DT.drop_duplicates()) # 한 환자의 청구항들의 날짜들
    combi_drugs = [0,0,0,0,0]
    for d in medi_date : # 날짜별 단위로 묶어서 약물 출력할 것임.
        one_day = one_person[one_person.MDCARE_STRT_DT == d]
        
        #### 어떤 병용 요법을 가장 오래 썼는지 통계를 낸다. 
        drugli = list(one_day.Drug_Type)
        for indx in range(len(drugli)):
            drugli[indx] = drugli[indx].split('_')

        ex_list = np.hstack(drugli)
        i = len(list(set(ex_list)))
        combi_drugs[i]+=np.min(list(one_day.TOT_MCNT)) # 한번 처방일에 준 약물 조합들 중 최소 처방일을 그 처방약을 다 먹은 기간으로 정의.
        ####
    
    minus_drug_group.loc[p] = np.argmax(combi_drugs)
    

print('병용개수   환자수')    
print(minus_drug_group.combination_n.value_counts().sort_index())



plus_group_table = patient_drug_group.combination_n.value_counts().sort_index()
plus_group_table.loc[0]=(len(list(patient_CV_outcome.RN_INDI.drop_duplicates())) 
                          - len(CV_plus_list))
plus_group_table =plus_group_table.sort_index()

total_table = pd.merge(plus_group_table,
        minus_drug_group.combination_n.value_counts().sort_index(), 
        left_index=True, right_index=True)
total_table = total_table.rename(columns={'combination_n_x':'CV+','combination_n_y':'CV-'})

total_table

Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)는 총 22 명 입니다.
Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)는 총 1 명 입니다. 

patient RN_INDI :  1144977.0
patient RN_INDI :  1442866.0
patient RN_INDI :  1727902.0
patient RN_INDI :  2067949.0
patient RN_INDI :  2251892.0
patient RN_INDI :  3162578.0
patient RN_INDI :  3667656.0
patient RN_INDI :  4418933.0
patient RN_INDI :  4534031.0
patient RN_INDI :  4753701.0
patient RN_INDI :  5508075.0
patient RN_INDI :  6028079.0
patient RN_INDI :  6684139.0
patient RN_INDI :  7167317.0
patient RN_INDI :  7338387.0
patient RN_INDI :  7491636.0
patient RN_INDI :  8268041.0
patient RN_INDI :  8343986.0
patient RN_INDI :  8755695.0
patient RN_INDI :  8953059.0
patient RN_INDI :  8981999.0
patient RN_INDI :  9640542.0
병용개수   환자수
0     14
1    123
2     67
3     12
Name: combination_n, dtype: int64


,CV+,CV-
0,31,14
1,14,123
2,6,67
3,2,12


In [21]:
'''
65세 이상
병용투여	CV outcome +	CV outcome - 
1    	     14	               123
2	          6            	    67
3	          2              	12
'''

table = [list(total_table.iloc[1]),
        list(total_table.iloc[2]),
        list(total_table.iloc[3])]

print(*table,sep ='\n')


chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)


[14, 123]
[6, 67]
[2, 12]
chi2 :  0.5510736055107363 p-Value: 0.7591644922348284


In [22]:
'''
65세 이상 

병용투여	CV outcome +	CV outcome - 
1+2+3	     	 22            202 
0	             31             14
'''


table = [list(np.array(total_table.iloc[1])+np.array(total_table.iloc[2])+np.array(total_table.iloc[3])),
        list(total_table.iloc[0])]

print(*table,sep ='\n')

chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)

odds_r = odds_ratio(table)

print("Odds Ratio : ", odds_r )

[22, 202]
[31, 14]
chi2 :  78.94660749685605 p-Value: 6.381142035569806e-19
Odds Ratio :  0.04918556371766209


In [ ]:
DM_plus_ID = list(patient_CV_outcome[(patient_CV_outcome.SICK_SYM1.str.startswith(DM_code,na=False)) | (patient_CV_outcome.SICK_SYM2.str.startswith(DM_code,na=False))].RN_INDI.drop_duplicates())+list(patient_CV_minus[(patient_CV_minus.SICK_SYM1.str.startswith(DM_code,na=False)) | (patient_CV_minus.SICK_SYM2.str.startswith(DM_code,na=False))].RN_INDI.drop_duplicates())
# 모집단을 바꿈. 
#m60_I10_main = m60_I10_main[m60_I10.RN_INDI.isin(DM_plus_ID)]  # : DM+
#m60_I10_main = m60_I10_main[~m60_I10.RN_INDI.isin(DM_plus_ID)] # : DM- 


In [33]:
DM_plus_ID = list(patient_CV_outcome[(patient_CV_outcome.SICK_SYM1.str.startswith(DM_code,na=False)) | (patient_CV_outcome.SICK_SYM2.str.startswith(DM_code,na=False))].RN_INDI.drop_duplicates())+list(patient_CV_minus[(patient_CV_minus.SICK_SYM1.str.startswith(DM_code,na=False)) | (patient_CV_minus.SICK_SYM2.str.startswith(DM_code,na=False))].RN_INDI.drop_duplicates())



In [66]:
# 위에서 한 분석을 모집단만 바꿔서 그대로 시행한다. 

CV_event_KEY = m20[(m20.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (m20.SICK_SYM2.str.startswith(CV_event_code,na=False))]

CV_outcome = m60_I10_main[m60_I10_main.RN_KEY.isin(list(CV_event_KEY.RN_KEY))] 

patient_CV_outcome = m60_I10_main[m60_I10_main.RN_INDI.isin(list(CV_outcome.RN_INDI))]

# 이 중 심혈관계 질병코드(outcome)발생이 관심약물 투여 이후인 환자번호(RN_INDI)를 이 리스트에 저장할 것임
CV_plus_list = [] # Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)
drug_plus_list = [] # outcome에 앞서 사용한 관심약물들을 저장한다.
Less_than_a_week = [] #Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)

CV_patients = list(patient_CV_outcome.RN_INDI.drop_duplicates()) #이 환자군을 돌아가면서 찾을건데
for p in CV_patients :
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    drug_first_time = one_person[one_person.Drug_Type.isin(Drug_Type_list)].MDCARE_STRT_DT.min() #관심약물을 먹은 최초일
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() #outcome 발병 최초일
    
    if drug_first_time < CV_first_time : #약을 처방한 시점이 앞서면 리스트에 저장한다.
        before_CV = one_person[one_person.MDCARE_STRT_DT < CV_first_time]
        
        medi_date = list(before_CV.MDCARE_STRT_DT.drop_duplicates()) 
        minimum_drugdate = 0 
        for d in medi_date : # 약의 처방일수가 7일 미만이면 제외한다.
            one_day = before_CV[before_CV.MDCARE_STRT_DT == d]
            minimum_drugdate+=np.max(list(one_day.TOT_MCNT))
        
        if minimum_drugdate < 7 : 
            Less_than_a_week.append(p)
            continue
        
        CV_plus_list.append(p)
        drug_plus_list.append(list(before_CV.Drug_Type.value_counts().index))
        
drug_plus_stat = patient_CV_outcome[patient_CV_outcome.RN_INDI.isin(CV_plus_list)].Drug_Type.value_counts()
  

print('Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)는 총', len(CV_plus_list), '명 입니다.') # 
print('Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)는 총', len(Less_than_a_week), '명 입니다.','\n') #

# 계속해서, 약물 사용의 변화를 알아본다. (CV+/ outcome이전 약물 사용 환자군)
patient_drug_group = pd.DataFrame({'CV_plus_RN_INDI': CV_plus_list, 'combination_n':0 }).set_index('CV_plus_RN_INDI') #여기에 환자별 병용요법 가지 수를 넣을 것임.

for p in CV_plus_list : 
    one_person = patient_CV_outcome[patient_CV_outcome.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.
    #outcome 발병 최초일
    CV_first_time = one_person[(one_person.SICK_SYM1.str.startswith(CV_event_code,na=False)) | (one_person.SICK_SYM2.str.startswith(CV_event_code,na=False))].MDCARE_STRT_DT.min() 
    # ! 조심 one_person 변수 새로이 할당됨.
    one_person = one_person[one_person.Drug_Type.isin(Drug_Type_list)] #관심약물이 있는 청구항만 고름.
    medi_date = list(one_person.MDCARE_STRT_DT.drop_duplicates()) # 한 환자의 청구항들의 날짜들
    
    
    combi_drugs = [0,0,0,0,0]
    
    print('patient RN_INDI : ',p)
    for d in medi_date : # 날짜별 단위로 묶어서 약물 출력할 것임.
        if d < CV_first_time :
            one_day = one_person[one_person.MDCARE_STRT_DT == d]

           
            #### 어떤 병용 요법을 가장 오래 썼는지 통계를 낸다. 
            drugli = list(one_day.Drug_Type)
            for indx in range(len(drugli)):
                drugli[indx] = drugli[indx].split('_')

            ex_list = np.hstack(drugli)
            i = len(list(set(ex_list)))
            combi_drugs[i]+=np.min(list(one_day.TOT_MCNT))  # 한번 처방일에 준 약물 조합들 중 최소 처방일을 그 처방약을 다 먹은 기간으로 정의.
            ####

        else:
            break
    
    patient_drug_group.loc[p] = np.argmax(combi_drugs)

        
'''CV+/약물 사용 환자 중에서 1,2,3제 요법을 사용한 환자의 수.'''
# 7일미만 사용자는 0으로 표시 
for i in range(len(Less_than_a_week)) :
    patient_drug_group.loc[Less_than_a_week[i]] = 0

patient_drug_group = patient_drug_group.sort_index()    

patient_CV_minus = m60_I10_main[~m60_I10_main.RN_INDI.isin(list(patient_CV_outcome.RN_INDI))] #409명. 26198건
# 관찰기간동안 CV outcome이 없는 환자.
CV_minus_list = list(patient_CV_minus.RN_INDI.drop_duplicates())

# 여기에 환자별 병용요법 가지 수를 넣을 것임.
minus_drug_group = pd.DataFrame({'CV_plus_RN_INDI': CV_minus_list, 'combination_n':0 }).set_index('CV_plus_RN_INDI') 


minus_patients = list(patient_CV_minus.RN_INDI.drop_duplicates()) #이 환자군을 돌아가면서 찾을건데
for p in minus_patients : 
    one_person = patient_CV_minus[patient_CV_minus.RN_INDI == p].sort_values(['MDCARE_STRT_DT']) # 한 환자(RN_INDI)만 추려냄.

    if ~(one_person.Drug_Type.isin(Drug_Type_list).any()) : # 관심 혈압강하제가 하나도 없으면 사용약물 0으로 하고 그냥 넘어감
        minus_drug_group.loc[p] = 0 
        continue
    
    one_person = one_person[one_person.Drug_Type.isin(Drug_Type_list)] #관심약물이 있는 청구항만 고름.
    medi_date = list(one_person.MDCARE_STRT_DT.drop_duplicates()) # 한 환자의 청구항들의 날짜들
    combi_drugs = [0,0,0,0,0]
    for d in medi_date : # 날짜별 단위로 묶어서 약물 출력할 것임.
        one_day = one_person[one_person.MDCARE_STRT_DT == d]
        
        #### 어떤 병용 요법을 가장 오래 썼는지 통계를 낸다. 
        drugli = list(one_day.Drug_Type)
        for indx in range(len(drugli)):
            drugli[indx] = drugli[indx].split('_')

        ex_list = np.hstack(drugli)
        i = len(list(set(ex_list)))
        combi_drugs[i]+=np.min(list(one_day.TOT_MCNT)) # 한번 처방일에 준 약물 조합들 중 최소 처방일을 그 처방약을 다 먹은 기간으로 정의.
        ####
    
    minus_drug_group.loc[p] = np.argmax(combi_drugs)
    

print('병용개수   환자수')    
print(minus_drug_group.combination_n.value_counts().sort_index())



plus_group_table = patient_drug_group.combination_n.value_counts().sort_index()
plus_group_table.loc[0]=(len(list(patient_CV_outcome.RN_INDI.drop_duplicates())) 
                          - len(CV_plus_list))
plus_group_table =plus_group_table.sort_index()

total_table = pd.merge(plus_group_table,
        minus_drug_group.combination_n.value_counts().sort_index(), 
        left_index=True, right_index=True)
total_table = total_table.rename(columns={'combination_n_x':'CV+','combination_n_y':'CV-'})

total_table

Outcome 발생이 관심약물 투여 이후인 환자번호(RN_INDI)는 총 20 명 입니다.
Outcome 발생이 관심약물 투여 이후지만 약물 투여일수가 7일 미만인 환자번호(RN_INDI)는 총 1 명 입니다. 

patient RN_INDI :  1144977.0
patient RN_INDI :  1442866.0
patient RN_INDI :  1727902.0
patient RN_INDI :  2251892.0
patient RN_INDI :  3162578.0
patient RN_INDI :  3667656.0
patient RN_INDI :  4418933.0
patient RN_INDI :  4534031.0
patient RN_INDI :  4685938.0
patient RN_INDI :  4753701.0
patient RN_INDI :  6028079.0
patient RN_INDI :  6684139.0
patient RN_INDI :  7167317.0
patient RN_INDI :  7338387.0
patient RN_INDI :  7491636.0
patient RN_INDI :  8268041.0
patient RN_INDI :  8343986.0
patient RN_INDI :  8755695.0
patient RN_INDI :  8953059.0
patient RN_INDI :  9640542.0
병용개수   환자수
0     38
1    165
2     92
3     17
Name: combination_n, dtype: int64


,CV+,CV-
0,40,38
1,13,165
2,6,92
3,1,17


In [70]:
#위 테이블은 당뇨가 없는 사람에서의 조사이다.
'''
당뇨가 없는 사람.
병용투여	CV outcome +	CV outcome - 
1+2+3	     	20             274
0	            40              38  
'''


table = [list(np.array(total_table.iloc[1])+np.array(total_table.iloc[2])+np.array(total_table.iloc[3])),
        list(total_table.iloc[0])]

print(*table,sep ='\n')

chi2, pvalue,dof_,_ = stats.chi2_contingency(table)
print("chi2 : ", chi2, "p-Value:", pvalue)

odds_r = odds_ratio(table)

print("Odds Ratio : ", odds_r )

[20, 274]
[40, 38]
chi2 :  86.8981111379463 p-Value: 1.142584425094399e-20
Odds Ratio :  0.06934306569343066


In [ ]:
#연습노트

데이터프레임명.loc[인덱스]
print('Outcome 발생이 관심약물 투여 이후인 환자에서 사용한 약물조합은 다음과 같습니다.')
print(*drug_plus_list, sep ='\n',)
#중복제거 print(*list(set([tuple(set(i)) for i in drug_plus_list])), sep ='\n')
m20[m20['SICK_SYM2']==b'I10'].drop_duplicates('RN_INDI',keep='first') #308명
m20[m20.SICK_SYM1.str.startswith(CV_event_code,na=False)] # 11372 개

m20['SICK_SYM2'].value_counts #부상병
#m60에서 위 55명 환자만 전체 이력을 뽑는다. 21129 건 
m60[m60.RN_INDI.isin(CV_outcome_RN_INDI)].sort_values('RN_INDI')
#CV_outcome이 있는 환자의 RN_INDI를 뽑는다. 55명
CV_outcome_RN_INDI = list(CV_outcome_drug.index.levels[0])